In [1]:
print('hello')

hello


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import upsetplot
import cerberus

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [11]:
config_file = '../snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [12]:
# procap meta
procap_meta = pd.read_csv('../'+expand(config['procap']['lib_meta'],
                                 species='human')[0],
                                 sep='\t')

In [14]:
meta_file = '../'+expand(config['procap']['meta'], species='human')[0]
biosamp_map_file = '../'+expand(config['ref']['biosamp_map'])[0]
lib_meta = '../'+expand(config['procap']['lib_meta'], species='human')[0]
cerberus_h5= '../'+expand(config['data']['cerb_annot'], species='human')[0]
ca_plus = '../'+expand(config['data']['ca_plus'], species='human')[0]
print(cerberus_h5)
print(ca_plus)

beds = expand(expand(config['procap']['bed'],
               zip,
               procap_dataset=procap_meta.dataset.unique().tolist(),
               allow_missing=True),
               species='human',
               output_type='bidirectional_peaks')
               # output_type=['bidirectional_peaks', 'unidirectional_peaks'])

../data/human/cerberus_annot.h5
../data/human/cerberus_annot_plus.h5


In [15]:
# ca = ceberus.read(ca_plus)

In [8]:
# ## updating cerberus to incorporate new bed files into cerberusannotation
ca = cerberus.read(cerberus_h5)

In [10]:
# for bed, dataset in zip(beds, procap_meta.dataset.unique().tolist()):
#     # import pdb; pdb.set_trace() # look how column makeup changes between these
#     bed = '../'+bed
#     if 'bidirectional' in bed:
#         source = f'{dataset}_bi'
#     else:
#         source = f'{dataset}_uni'
#     print(bed)
#     print(f'Adding TSSs from {source}')
#     ca.add_bed(bed, False, False, source, 'tss', slack=20)
# # ca.add_bed(bed, False, True, 'procap_k562_bi', 'tss', slack=20) # this doesn't work for some reason


In [8]:
# ca.write('test_ca_add_bed.h5')

In [9]:
# ca.tss.source.unique()

In [10]:
# try using classical agg_ends 
ca = cerberus.read(cerberus_h5)
ca_tss_fname = 'test_ca_tss.bed'
pr.PyRanges(ca.tss).to_bed(ca_tss_fname)
files = [ca_tss_fname]+['../'+bed for bed in beds]
add_ends = [True]+[False for i in range(len(beds))]
refs = [True]+[False for i in range(len(beds))]
sources = ['cerberus']+[f'{dataset}_bi' for dataset in procap_meta.dataset.unique().tolist()]
mode = 'tss'
slack = 20

# ofile = 'test_ca_new_agg_ends.bed' # for rewrite
ofile = 'test_ca_old_agg_ends.bed'

cerberus.agg_ends(files,
         add_ends,
         refs,
         sources,
         mode,
         slack,
         ofile)

In [75]:
# print(len(files))
# print(len(add_ends))
# print(len(refs))
# print(len(sources))

In [76]:
## how do results compare?

In [11]:
bed2 = pr.read_bed(ofile).as_df().rename({'ThickStart':'source',
                                          'ThickEnd':'novelty'},
                                         axis=1)

In [12]:
len(bed2.index)

153941

In [13]:
bed2.loc[bed2.source.str.contains('bi')]
# cl_tss = bed2.loc[bed2.source.str.contains('_bi')]
cl_tss = bed2.copy(deep=True)

In [14]:
# bed2.loc[bed2.Name=='ENSG00000000003_1'].ThickStart.values[0]

In [15]:
ca = cerberus.read('test_ca_add_bed.h5')

In [16]:
len(ca.tss.index)
oo_tss = ca.tss.copy(deep=True)

In [17]:
cl_tss.sort_values(by=['Name']).head()

,Chromosome,Start,End,Name,Score,Strand,source,novelty
150685,chrX,100636638,100636856,ENSG00000000003_1,.,-,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known
150686,chrX,100637053,100637154,ENSG00000000003_2,.,-,"v40,v29,lapa,pls,mcf10a_1_bi,caco2_1_bi,huvec_...",Known
150687,chrX,100639940,100640041,ENSG00000000003_3,.,-,"v40,v29,fantom_cage",Known
148114,chrX,100584885,100584986,ENSG00000000005_1,.,+,"v40,fantom_cage,encode_rampage,pls",Known
148115,chrX,100593573,100593674,ENSG00000000005_2,.,+,"v40,v29,lapa,encode_cage,fantom_cage,encode_ra...",Known


In [18]:
oo_tss.sort_values(by=['Name']).head()

,Chromosome,Start,End,Strand,Name,source,novelty,gene_id,tss
151994,chrX,100636638,100636856,-,ENSG00000000003_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000003,1
151995,chrX,100637053,100637154,-,ENSG00000000003_2,"v40,v29,lapa,pls,mcf10a_1_bi,caco2_1_bi,huvec_...",Known,ENSG00000000003,2
152362,chrX,100639940,100640041,-,ENSG00000000003_3,"v40,v29,fantom_cage",Known,ENSG00000000003,3
149795,chrX,100584885,100584986,+,ENSG00000000005_1,"v40,fantom_cage,encode_rampage,pls",Known,ENSG00000000005,1
149796,chrX,100593573,100593674,+,ENSG00000000005_2,"v40,v29,lapa,encode_cage,fantom_cage,encode_ra...",Known,ENSG00000000005,2


In [19]:
all_tss = oo_tss.merge(cl_tss, how='outer',
                       on=['Chromosome', 'Start', 'End', 'Strand', 'Name', 'source', 'novelty'],
                       suffixes=('_oo', '_cl'))

In [20]:
len(all_tss)

153941

In [21]:
all_tss.loc[all_tss.Name.duplicated(keep=False)].sort_values(by='Name')

,Chromosome,Start,End,Strand,Name,source,novelty,gene_id,tss,Score


In [22]:
# all_tss.loc[all_tss.Name=='ENSG00000000003_1', 'source'].values
print(oo_tss.loc[oo_tss.Name=='ENSG00000000003_1', 'source'].values)
print(cl_tss.loc[cl_tss.Name=='ENSG00000000003_1', 'source'].values)

['v40,v29,lapa,gtex,encode_cage,fantom_cage,encode_rampage,pls,pels,mcf10a_1_bi,caco2_1_bi,huvec_1_bi,calu3_1_bi,a673_1_bi']
['v40,v29,lapa,gtex,encode_cage,fantom_cage,encode_rampage,pls,pels,mcf10a_1_bi,caco2_1_bi,huvec_1_bi,calu3_1_bi,a673_1_bi']


In [ ]:
oo_tss.loc[

In [29]:
# ca.tss.loc[ca.tss.source.str.contains('bi')].source.tolist()[:5]
# ca.tss.loc[ca.tss.source.str.contains('bi')]
# oo_tss = ca.tss.loc[ca.tss.source.str.contains('_bi')]

In [30]:
len(oo_tss.index)

39059

In [34]:
oo_tss.head()

,Chromosome,Start,End,Strand,Name,source,novelty,gene_id,tss
151994,chrX,100636638,100636856,-,ENSG00000000003_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000003,1
151995,chrX,100637053,100637154,-,ENSG00000000003_2,"v40,v29,lapa,pls,mcf10a_1_bi,caco2_1_bi,huvec_...",Known,ENSG00000000003,2
142158,chr20,50958366,50958605,-,ENSG00000000419_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000419,1
142445,chr20,50959089,50959190,-,ENSG00000000419_2,"v40,fantom_cage,pls,mcf10a_1_bi,caco2_1_bi,cal...",Known,ENSG00000000419,2
7250,chr1,169893845,169894009,-,ENSG00000000457_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000457,1


In [35]:
cl_tss.head()

,Chromosome,Start,End,Name,Score,Strand,source,novelty
0,chr1,169794989,169795129,ENSG00000000460_1,.,+,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known
2,chr1,169794679,169794780,ENSG00000000460_3,.,+,"v40,v29,lapa,gtex,pls,mcf10a_1_bi,k562_4_bi,ca...",Known
3,chr1,169795870,169795971,ENSG00000000460_4,.,+,"v40,v29,pls,huvec_1_bi,calu3_1_bi",Known
5,chr1,196651703,196652106,ENSG00000000971_1,.,+,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known
11,chr1,24415751,24415908,ENSG00000001461_1,.,+,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known


In [17]:
ca = cerberus.read(ca_plus)
df = ca.tss.copy(deep=True)

In [20]:
df = upsetplot.from_memberships(df.source.str.split(','), data=df).reset_index()


In [21]:
df.head()

,_bia673_1,_bicaco2_1,_bicalu3_1,_bihuvec_1,_bik562_1,_bik562_2,_bik562_3,_bik562_4,_bik562_5,_bik562_6,...,v40,Chromosome,Start,End,Strand,Name,source,novelty,gene_id,tss
0,True,True,True,True,False,False,False,False,False,False,...,True,chrX,100636638,100636856,-,ENSG00000000003_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000003,1
1,True,True,True,True,False,False,False,False,False,False,...,True,chrX,100637053,100637154,-,ENSG00000000003_2,"v40,v29,lapa,pls,_bimcf10a_1,_bicaco2_1,_bihuv...",Known,ENSG00000000003,2
2,False,False,False,False,False,False,False,False,False,False,...,True,chrX,100639940,100640041,-,ENSG00000000003_3,"v40,v29,fantom_cage",Known,ENSG00000000003,3
3,False,False,False,False,False,False,False,False,False,False,...,True,chrX,100584885,100584986,+,ENSG00000000005_1,"v40,fantom_cage,encode_rampage,pls",Known,ENSG00000000005,1
4,False,False,False,False,False,False,False,False,False,False,...,True,chrX,100593573,100593674,+,ENSG00000000005_2,"v40,v29,lapa,encode_cage,fantom_cage,encode_ra...",Known,ENSG00000000005,2


In [23]:
columns = ['encode_cage', 'encode_rampage', 'fantom_cage', 'gtex', 'v29', 'v40']
columns_2 = [c for c in df.columns if '_bi' in c]
columns_2 += [c for c in df.columns if '_uni' in c]
len(df.loc[(df[columns].sum(axis=1)==0)*(df['lapa']==True)&(df[columns_2].sum(axis=1)>0)].index)

2056

In [37]:
len(df.loc[df[columns].sum(axis=1)>0].index)
# df.loc[df[columns].sum(axis=1)>0].head()

147150

In [7]:
# somehow make sure this is working correctly.... 
# * check to make sure that the tss names haven't changed (ie same region in a and b)
# * check to make sure htat this still works from the cmd line

,Chromosome,Start,End,Strand,Name,source,novelty,gene_id,tss
151994,chrX,100636638,100636856,-,ENSG00000000003_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000003,1
151995,chrX,100637053,100637154,-,ENSG00000000003_2,"v40,v29,lapa,pls,mcf10a_1_bi,caco2_1_bi,huvec_...",Known,ENSG00000000003,2
152362,chrX,100639940,100640041,-,ENSG00000000003_3,"v40,v29,fantom_cage",Known,ENSG00000000003,3
149795,chrX,100584885,100584986,+,ENSG00000000005_1,"v40,fantom_cage,encode_rampage,pls",Known,ENSG00000000005,1
149796,chrX,100593573,100593674,+,ENSG00000000005_2,"v40,v29,lapa,encode_cage,fantom_cage,encode_ra...",Known,ENSG00000000005,2


In [ ]:
pr.PyRanges(ca.tss).to_bed('test_tss.bed')

In [14]:
pr.PyRanges(ca.tss_map).to_bed('test_tss_map.bed')

In [11]:
len(ca.tss.loc[ca.tss.source=='lapa,procap_k562_bi'])

42

In [19]:
ca.tss['coord'] = ca.tss.Chromosome.astype(str)+':'+ca.tss.Start.astype(str)+'-'+ca.tss.End.astype(str)
ca.tss.loc[ca.tss.source=='lapa,procap_k562_bi']

,Chromosome,Start,End,Strand,Name,source,novelty,coord
25283,chr3,100262670,100262771,+,ENSG00000036054_7,"lapa,procap_k562_bi",Novel,chr3:100262670-100262771
123352,chr17,7235112,7235213,-,ENSG00000040633_8,"lapa,procap_k562_bi",Novel,chr17:7235112-7235213
89241,chr12,51059936,51060037,+,ENSG00000050426_9,"lapa,procap_k562_bi",Novel,chr12:51059936-51060037
51482,chr6,170580051,170580152,-,ENSG00000071994_5,"lapa,procap_k562_bi",Novel,chr6:170580051-170580152
19839,chr2,61483039,61483140,-,ENSG00000082898_28,"lapa,procap_k562_bi",Novel,chr2:61483039-61483140
93482,chr12,120196760,120196861,-,ENSG00000089157_6,"lapa,procap_k562_bi",Novel,chr12:120196760-120196861
119070,chr17,18209770,18209871,+,ENSG00000091542_6,"lapa,procap_k562_bi",Novel,chr17:18209770-18209871
131037,chr19,14052436,14052537,+,ENSG00000104998_2,"lapa,procap_k562_bi",Novel,chr19:14052436-14052537
34285,chr4,169722932,169723033,+,ENSG00000109572_9,"lapa,procap_k562_bi",Novel,chr4:169722932-169723033
89273,chr12,111448802,111448903,+,ENSG00000111252_9,"lapa,procap_k562_bi",Novel,chr12:111448802-111448903


In [21]:
ca.tss_map.head()
ca.tss_map.loc[ca.tss_map.Name=='ENSG00000040633_8']

,Chromosome,Start,End,Strand,source,Name
2017557,chr17,7235112,7235213,-,lapa,ENSG00000040633_8
123352,chr17,7235213,7235483,-,procap_k562_bi,ENSG00000040633_8


In [4]:
# def get_lib_meta_from_enc_meta(meta_file, 
#                                biosamp_map_file,
#                                ofile):
    
#     df = pd.read_csv(meta_file, sep='\t')    

#     # get only one output type so that you don't have multiple entries per thing
#     df = df.loc[df['Output type'] == df['Output type'].tolist()[0]]

#     df = df[['File accession', 'Experiment accession', 'Biosample term name', 'Biosample type', 'Technical replicate(s)', 'Biological replicate(s)']]
#     df['classification'] = 'cell_line'
#     df.loc[df['Biosample type']=='tissue', 'classification'] = 'tissue'
#     df = convert_encode_desc(df, 'Biosample term name')

#     # convert hyphenated cell line names
#     term_map = pd.read_csv(biosamp_map_file, sep='\t',
#                            header=None, names=['eid', 'old_name', 'idk1', 'idk2', 'new_name'])
#     term_map = convert_encode_desc(term_map, 'old_name')
#     term_map = convert_encode_desc(term_map, 'new_name')
#     term_map = term_map[['old_name', 'new_name']]
#     term_map.drop_duplicates(inplace=True)
#     n1 = len(df.index)
#     # print(n1)
#     df = df.merge(term_map, how='left', left_on='Biosample term name', right_on='old_name')
#     n2 = len(df.index)
#     # print(n2)
#     if n1 != n2:
#         print('Duplicated thingies, check for DE samples')
#     df.rename({'Biosample term name': 'sample'}, axis=1, inplace=True)
#     df.loc[~df.new_name.isnull(), 'sample'] = df.loc[~df.new_name.isnull(), 'new_name']

#     # df.loc[df['Experiment accession'].duplicated(keep=False)].sort_values(by='Experiment accession')
#     # df.loc[df['sample']!=df['new_name']]
#     temp = df[['Experiment accession', 'sample', 'File accession']].groupby(['Experiment accession', 'sample']).count().reset_index()
#     temp['biorep'] = temp.groupby('sample').cumcount()+1
#     temp = temp[['Experiment accession', 'biorep']]
#     temp.biorep = temp.biorep.astype(str)
#     df = df.merge(temp, on='Experiment accession')
#     # df['techrep'] = df.groupby('Experiment accession').cumcount()+1
#     df['dataset'] = df['sample']+'_'+df.biorep#+'_'+df.techrep.astype(str)

#     # get the different output type files and merge in
#     cols = ['Experiment accession', 'sample', 'classification', 'Technical replicate(s)', 'dataset']
#     df = df[cols]

#     df2 = pd.read_csv(meta_file, sep='\t')    
#     df2 = df2[['File accession', 'Experiment accession', 'Output type']]

#     df2 = df2[['Experiment accession',
#                'File accession',
#                'Output type']].pivot(values='File accession',
#                                      index='Experiment accession',
#                                      columns='Output type')

#     df2.columns.name = ''
#     df2.reset_index(inplace=True)
#     df = df.merge(df2, how='left', on='Experiment accession')
#     df.to_csv(ofile, sep='\t', index=False)

In [5]:
get_lib_meta_from_enc_meta(meta_file, 
                           biosamp_map_file,
                           lib_meta)

In [66]:
#     df = pd.read_csv(meta_file, sep='\t')    

#     # get only one output type so that you don't have multiple entries per thing
#     df = df.loc[df['Output type'] == df['Output type'].tolist()[0]]

#     df = df[['File accession', 'Experiment accession', 'Biosample term name', 'Biosample type', 'Technical replicate(s)', 'Biological replicate(s)']]
#     df['classification'] = 'cell_line'
#     df.loc[df['Biosample type']=='tissue', 'classification'] = 'tissue'
#     df = convert_encode_desc(df, 'Biosample term name')

#     # convert hyphenated cell line names
#     term_map = pd.read_csv(biosamp_map_file, sep='\t',
#                            header=None, names=['eid', 'old_name', 'idk1', 'idk2', 'new_name'])
#     term_map = convert_encode_desc(term_map, 'old_name')
#     term_map = convert_encode_desc(term_map, 'new_name')
#     term_map = term_map[['old_name', 'new_name']]
#     term_map.drop_duplicates(inplace=True)
#     n1 = len(df.index)
#     # print(n1)
#     df = df.merge(term_map, how='left', left_on='Biosample term name', right_on='old_name')
#     n2 = len(df.index)
#     # print(n2)
#     if n1 != n2:
#         print('Duplicated thingies, check for DE samples')
#     df.rename({'Biosample term name': 'sample'}, axis=1, inplace=True)
#     df.loc[~df.new_name.isnull(), 'sample'] = df.loc[~df.new_name.isnull(), 'new_name']

#     # df.loc[df['Experiment accession'].duplicated(keep=False)].sort_values(by='Experiment accession')
#     # df.loc[df['sample']!=df['new_name']]
#     temp = df[['Experiment accession', 'sample', 'File accession']].groupby(['Experiment accession', 'sample']).count().reset_index()
#     temp['biorep'] = temp.groupby('sample').cumcount()+1
#     temp = temp[['Experiment accession', 'biorep']]
#     temp.biorep = temp.biorep.astype(str)
#     df = df.merge(temp, on='Experiment accession')
#     # df['techrep'] = df.groupby('Experiment accession').cumcount()+1
#     df['dataset'] = df['sample']+'_'+df.biorep#+'_'+df.techrep.astype(str)

#     # get the different output type files and merge in
#     cols = ['Experiment accession', 'sample', 'classification', 'Technical replicate(s)', 'dataset']
#     df = df[cols]

#     df2 = pd.read_csv(meta_file, sep='\t')    
#     df2 = df2[['File accession', 'Experiment accession', 'Output type']]

#     df2 = df2[['Experiment accession',
#                'File accession',
#                'Output type']].pivot(values='File accession',
#                                      index='Experiment accession',
#                                      columns='Output type')

#     df2.columns.name = ''
#     df2.reset_index(inplace=True)
#     df = df.merge(df2, how='left', on='Experiment accession')
#     df.to_csv(ofile, sep='\t', index=False)

In [55]:
# df.loc[df['Experiment accession'].duplicated(keep=False)].sort_values(by='Experiment accession')
# df.loc[df['sample']!=df['new_name']]
temp = df[['Experiment accession', 'sample', 'File accession']].groupby(['Experiment accession', 'sample']).count().reset_index()
temp['biorep'] = temp.groupby('sample').cumcount()+1
temp = temp[['Experiment accession', 'biorep']]
temp.biorep = temp.biorep.astype(str)
df = df.merge(temp, on='Experiment accession')
# df['techrep'] = df.groupby('Experiment accession').cumcount()+1
df['dataset'] = df['sample']+'_'+df.biorep#+'_'+df.techrep.astype(str)

In [57]:
# get the different output type files and merge in
cols = ['Experiment accession', 'sample', 'classification', 'Technical replicate(s)', 'dataset']
df = df[cols]

In [58]:
df2 = pd.read_csv(meta_file, sep='\t')    
df2 = df2[['File accession', 'Experiment accession', 'Output type']]

In [60]:
df2 = df2[['Experiment accession',
           'File accession',
           'Output type']].pivot(values='File accession',
                                 index='Experiment accession',
                                 columns='Output type')

In [61]:
df2.columns.name = ''
df2.reset_index(inplace=True)
df = df.merge(df2, how='left', on='Experiment accession')
df.to_csv(lib_meta, sep='\t', index=False)